### Accurate Uncertainties for Deep Learning Using Calibrated Regression
**Anthony Rentsch and Abhimanyu Vasishth**

An implementation and overview of the [2018 paper](https://arxiv.org/abs/1807.00263) by Kuleshov et al.

---

In [14]:
# packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

#### Problem Statement 

what is the problem the paper aims to solve?

#### Background 

why is this problem important or interesting?

#### Existing work 

what has been done in literature?

In [16]:
# Platt Scaling
X,y = make_classification(n_samples=10000, n_features=2, n_redundant=0, random_state=207)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=207, train_size=0.8)

logreg_model = LogisticRegression(solver='lbfgs').fit(X_train, y_train)

#### Contribution 

what is gap in literature that the paper is trying to fill? What is the unique contribution

#### Technical content 

high level - what are the high level ideas behind their technical contribution
details - highlight (not copy and paste entire sections) the relevant details that are important to focus on (e.g. if there's a model, define it; if there is a theorem, state it and explain why it's important, etc).

#### Experiments 

which types of experiments were performed? What claims were these experiments trying to prove? Did the results prove the claims?

#### Evaluation 

(your opinion) - do you think the work is technically sound? Do you think the proposed model/inference method is practical to use on real data and tasks? Do you think the experimental section was strong (there are sufficient evidence to support the claims and eliminate confounding factors)?

#### Future work 

(for those interested in continuing research in a related field) - do you think you can suggest a concrete change or modification that would improve the existing solution(s) to the problem of interest? Try to implement some of these changes/modifications.

#### Code

- At least one clear working pedagogical example demonstrating the problem the paper is claiming to solve. 
- At lease a bare bones implementation of the model/algorithm/solution (in some cases, you may be able to make assumptions  to simplify the model/algorithm/solution with the approval of your instructor)
- Demonstration on at least one instance that your implementation solves the problem.
- Demonstration on at least one instance the failure mode of the model/algorithm/solution, with an explanation for why failure occurred (is the dataset too large? Did you choose a bad hyper parameter?). The point of this is to point out edge cases to the user.

---